In [2]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
from PIL import Image
import cv2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_csv("Task4.csv")
df

Unnamed: 0  Epochs  Batch Size  Learning Rate  Dropout Rate  FC1 Nodes  \
0             0       3          32          0.001           0.2         10   
1             1       6          32          0.001           0.2         10   
2             2       9          32          0.001           0.2         10   
3             3       3          32          0.001           0.2         30   
4             4       6          32          0.001           0.2         30   
..          ...     ...         ...            ...           ...        ...   
103         103       6         256          0.010           1.0         30   
104         104       9         256          0.010           1.0         30   
105         105       3         256          0.010           1.0         50   
106         106       6         256          0.010           1.0         50   
107         107       9         256          0.010           1.0         50   

      Accuracy                                       train_losses  \
0    65.699997  [2.3779895305633545, 2.298424482345581, 2.2997...   
1    71.620003  [2.3779895305633545, 2.298424482345581, 2.2997...   
2    74.900002  [2.3779895305633545, 2.298424482345581, 2.2997...   
3    72.269997  [2.3386623859405518, 2.3343656063079834, 2.347...   
4    75.629997  [2.3386623859405518, 2.3343656063079834, 2.347...   
..         ...                                                ...   
103  10.230000  [2.3041088581085205, 2.3246684074401855, 2.309...   
104  10.210000  [2.3041088581085205, 2.3246684074401855, 2.309...   
105  11.460000  [2.303673267364502, 2.3031933307647705, 2.3070...   
106  11.860000  [2.303673267364502, 2.3031933307647705, 2.3070...   
107  12.110000  [2.303673267364502, 2.3031933307647705, 2.3070...   

                                         train_counter  \
0    [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
1    [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
2    [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
3    [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
4    [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
..                                                 ...   
103  [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
104  [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
105  [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
106  [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   
107  [0, 640, 1280, 1920, 2560, 3200, 3840, 4480, 5...   

                                           test_losses  \
0    [2.3217968017578126, 1.7890741333007814, 1.284...   
1    [2.3217968017578126, 1.7890741333007814, 1.284...   
2    [2.3217968017578126, 1.7890741333007814, 1.284...   
3    [2.312999560546875, 1.1253486328125, 0.8473098...   
4    [2.312999560546875, 1.1253486328125, 0.8473098...   
..                                                 ...   
103  [2.312999560546875, 2.30888603515625, 2.307604...   
104  [2.312999560546875, 2.30888603515625, 2.307604...   
105  [2.3021302978515625, 2.3005751708984374, 2.300...   
106  [2.3021302978515625, 2.3005751708984374, 2.300...   
107  [2.3021302978515625, 2.3005751708984374, 2.300...   

                                          test_counter  
0    [0, 60000, 120000, 180000, 240000, 300000, 360...  
1    [0, 60000, 120000, 180000, 240000, 300000, 360...  
2    [0, 60000, 120000, 180000, 240000, 300000, 360...  
3    [0, 60000, 120000, 180000, 240000, 300000, 360...  
4    [0, 60000, 120000, 180000, 240000, 300000, 360...  
..                                                 ...  
103  [0, 60000, 120000, 180000, 240000, 300000, 360...  
104  [0, 60000, 120000, 180000, 240000, 300000, 360...  
105  [0, 60000, 120000, 180000, 240000, 300000, 360...  
106  [0, 60000, 120000, 180000, 240000, 300000, 360...  
107  [0, 60000, 120000, 180000, 240000, 300000, 360...  

[108 rows x 11 columns]

In [24]:
def meanGB(df):
    l = ["Epochs", "Batch Size", "Learning Rate", "Dropout Rate", "FC1 Nodes"]
    for i in l:
        print(df.groupby(by = i)["Accuracy"].mean(), "\n")
        print(df.groupby(by = i)["Accuracy"].median(), "\n")

In [25]:
meanGB(df)

Epochs
3    44.325278
6    50.091944
9    52.657500
Name: Accuracy, dtype: float64 

Epochs
3    51.139999
6    66.599998
9    70.260002
Name: Accuracy, dtype: float64 

Batch Size
32     55.107037
256    42.942777
Name: Accuracy, dtype: float64 

Batch Size
32     73.835003
256    43.809999
Name: Accuracy, dtype: float64 

Learning Rate
0.001    42.880926
0.010    55.168888
Name: Accuracy, dtype: float64 

Learning Rate
0.001    43.809999
0.010    74.160000
Name: Accuracy, dtype: float64 

Dropout Rate
0.2    69.049721
0.5    67.380556
1.0    10.644445
Name: Accuracy, dtype: float64 

Dropout Rate
0.2    74.730000
0.5    73.835003
1.0    10.445000
Name: Accuracy, dtype: float64 

FC1 Nodes
10    43.488611
30    50.777499
50    52.808611
Name: Accuracy, dtype: float64 

FC1 Nodes
10    52.350000
30    68.630001
50    69.200001
Name: Accuracy, dtype: float64 

